## Import libraries

In [30]:
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, Adadelta
from keras.models import model_from_json
from keras import backend as K
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

from sklearn.metrics import classification_report, confusion_matrix
from textblob import TextBlob

import numpy as np, string, pickle, warnings, random
import matplotlib.pyplot as plt

from IPython.display import display, HTML

from deepexplain.tensorflow import DeepExplain
import tensorflow as tf
warnings.filterwarnings("ignore")

## Download data

In [31]:
topWords = 50000
maxWords = 200
imdbDataPicklePath = './data/imdbData.pickle'
downloadFlag = 0

df = pd.read_csv('./data/train.csv')
x_train = df['utterance']
y_train = df['intent']

df = pd.read_csv('./data/val.csv')
x_val = df['utterance']
y_val = df['intent']

df = pd.read_csv('./data/test.csv')
x_test = df['utterance']
y_test = df['intent']

classes_train = set(y_train)
classes_val = set(y_val)
classes_test = set(y_test)
classes = classes_train.union(classes_val, classes_test)
nb_classes = len(classes)
sentimentDict = {}
for i, cl in enumerate(classes):
    sentimentDict[i] = cl
sentimentDict2 = {v: k for k, v in sentimentDict.items()}
print(sentimentDict)
y_train = [sentimentDict2[cl] for cl in list(y_train)]
y_val = [sentimentDict2[cl] for cl in list(y_val)]
y_test = [sentimentDict2[cl] for cl in list(y_test)]

data = x_train.append(x_val)
data = data.append(x_test)
vocab = []
for utter in data:
    for word in utter.split(' '):
        vocab.append(word)
vocab = set(vocab)
word2Index = {}
for i, word in enumerate(vocab):
    word2Index[word] = i
index2Word = {v: k for k, v in word2Index.items()}

train = []
for utter in x_train:
    utt = [word2Index[word] for word in utter.split(' ')]
    train.append(utt)
x_train = train
val = []
for utter in x_val:
    utt = [word2Index[word] for word in utter.split(' ')]
    val.append(utt)
x_val = val
test = []
for utter in x_test:
    utt = [word2Index[word] for word in utter.split(' ')]
    test.append(utt)
x_test = test

{0: 'ground_service', 1: 'airline', 2: 'ground_fare', 3: 'flight;atis_airline', 4: 'meal', 5: 'restriction', 6: 'quantity', 7: 'flight_no;atis_airline', 8: 'cheapest', 9: 'airfare;atis_flight_time', 10: 'aircraft', 11: 'day_name', 12: 'airline;atis_flight_no', 13: 'airfare', 14: 'flight', 15: 'distance', 16: 'flight_no', 17: 'airfare;atis_flight', 18: 'ground_service;atis_ground_fare', 19: 'airport', 20: 'abbreviation', 21: 'aircraft;atis_flight;atis_flight_no', 22: 'flight;atis_airfare', 23: 'capacity', 24: 'flight_time', 25: 'city'}


In [32]:
print(data)

0      i want to fly from baltimore to dallas round t...
1      round trip fares from baltimore to philadelphi...
2      show me the flights arriving on baltimore on j...
3      what are the flights which depart from san fra...
4      which airlines fly from boston to washington d...
                             ...                        
888    please find all the flights from cincinnati to...
889    find me a flight from cincinnati to any airpor...
890    i'd like to fly from miami to chicago on ameri...
891    i would like to book a round trip flight from ...
892    find me a flight that flies from memphis to ta...
Name: utterance, Length: 5871, dtype: object


In [46]:
stopWords = []

def getWordsFromIndexList(indexList):
    wordList = []
    for index in indexList:
        wordList.append(index2Word[index])

    return " ".join(wordList)

def getSentiment(predictArray):
    pred = int(predictArray[0])
    return sentimentDict[pred]

def getIndexFromWordList(wordList):
    indexList = []
    for word in wordList:
        print(word)
        indexList.append(str(word2Index[word]))
        
    return indexList

In [34]:
print (len(word2Index))

951


## Preprocess data

In [35]:
stopIndexList = []

# for stopWord in stopWords:
#     stopIndexList.append(word2Index[stopWord])

trainData = []

for indexList in x_train:
    processedList = [index for index in indexList]# if index not in stopIndexList]
    trainData.append(processedList)
    
x_train = trainData

## Data Pre-processing

In [36]:
'''
Padding data to keep vectors of same size
If size < 100 then it will be padded, else it will be cropped
'''
trainX = pad_sequences(x_train, maxlen = maxWords, value = 0.)
valX = pad_sequences(x_val, maxlen = maxWords, value = 0.)
testX = pad_sequences(x_test, maxlen = maxWords, value = 0.)

'''
One-hot encoding for the classes
'''
trainY = np_utils.to_categorical(y_train, num_classes = nb_classes)
valY = np_utils.to_categorical(y_val, num_classes = nb_classes)
testY = np_utils.to_categorical(y_test, num_classes = nb_classes)

## Network Parameters

In [37]:
p_W, p_U, weight_decay = 0, 0, 0
regularizer = l2(weight_decay) if weight_decay else None
sgdOptimizer = 'adam'
lossFun='categorical_crossentropy'
batchSize=25
numEpochs = 5
numHiddenNodes = 256

## Network Architecture

In [38]:
model = Sequential()
model.add(Embedding(topWords, numHiddenNodes, name='embedding_layer'))

model.add(Bidirectional(LSTM(numHiddenNodes, return_sequences=True,
                            recurrent_regularizer=regularizer, kernel_regularizer=regularizer,
                            bias_regularizer=regularizer, recurrent_dropout=p_W, dropout=p_U),
                        merge_mode='concat', name='bidi_lstm_layer'))
model.add(LSTM(numHiddenNodes, name = 'lstm_layer'))
model.add(Dropout(0.5, name = 'dropout'))

model.add(Dense(numHiddenNodes, name='dense_1'))
model.add(Dense(nb_classes, name='dense_2'))
model.add(Activation("softmax"))
# adam = Adam(lr=0.0001)
#adadelta = Adadelta(lr=0.01, rho=0.95, epsilon=1e-08)
model.compile(loss=lossFun, optimizer=sgdOptimizer, metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 256)         12800000  
_________________________________________________________________
bidi_lstm_layer (Bidirection (None, None, 512)         1050624   
_________________________________________________________________
lstm_layer (LSTM)            (None, 256)               787456    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 26)                6682      
_________________________________________________________________
activation_1 (Activation)    (None, 26)                0         
Total para

## Training the model

In [39]:
model.fit(trainX, trainY, batch_size=batchSize, epochs=numEpochs, verbose=1, validation_data=(valX, valY))

Train on 4478 samples, validate on 500 samples
Epoch 1/5
4478/4478 [==============================] - 316s - loss: 0.9493 - acc: 0.7814 - val_loss: 0.6772 - val_acc: 0.8420
Epoch 2/5
4478/4478 [==============================] - 323s - loss: 0.4035 - acc: 0.8973 - val_loss: 0.3570 - val_acc: 0.9220
Epoch 3/5
4478/4478 [==============================] - 328s - loss: 0.2194 - acc: 0.9433 - val_loss: 0.2817 - val_acc: 0.9400
Epoch 4/5
4478/4478 [==============================] - 498s - loss: 0.1433 - acc: 0.9685 - val_loss: 0.2180 - val_acc: 0.9580
Epoch 5/5
4478/4478 [==============================] - 529s - loss: 0.1177 - acc: 0.9723 - val_loss: 0.2134 - val_acc: 0.9620


# Model accuracy

In [40]:
score = model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

acc: 93.39%


In [41]:
predY = model.predict_classes(testX)
yPred = np_utils.to_categorical(predY, num_classes = nb_classes)
print("Classification Report:\n")
print(classification_report(testY, yPred))

893/893 [==============================] - 11s    
Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.97      0.93        36
           1       0.97      0.95      0.96        38
           2       1.00      0.86      0.92         7
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         0
           6       0.38      1.00      0.55         3
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.86      0.67      0.75         9
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         0
          13       0.96      0.98      0.97        48
          14       0.96      0.99      0.97       632
          15       0.86      0.60      0.71        10
      

## Save the Tensorflow Model

In [16]:
'''
Serialize model to JSON
'''
model_json = model.to_json()
with open("models/imdb_bi_lstm_big_tensorflow_model_ALL.json", "w") as json_file:
    json_file.write(model_json)

'''
Serialize weights to HDF5
'''
model.save_weights("models/imdb_bi_lstm_big_tensorflow_model_ALL.h5", overwrite=True)
print("Saved model to disk...")

FileNotFoundError: [Errno 2] No such file or directory: 'models/imdb_bi_lstm_big_tensorflow_model_ALL.json'

## Load the Tensorflow Model

In [ ]:
'''
Load json and create model
'''
modelNum=1
if modelNum is 1:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model.json', 'r')
elif modelNum is 2:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model_NOT.json', 'r')
elif modelNum is 3:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model_ALL.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

'''
Load weights into new model
'''
if modelNum is 1:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model.h5")
elif modelNum is 2:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model_NOT.h5")
elif modelNum is 3:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model_ALL.h5")

print ("Loading model from disk...")
model = loaded_model
print(model.summary())

## Testing the model

In [42]:
num = 120
num_next = num + 1
print("Testing for test case..." + str(num))
groundTruth = testY[num]
print(groundTruth)
print(len(testY))
print

sampleX = testX[num:num_next]
predictionClass = model.predict_classes(sampleX, verbose=0)
prediction = np_utils.to_categorical(predictionClass, num_classes = nb_classes)[0]

print("Text: " + str(getWordsFromIndexList(x_test[num-1])))
print("\nPrediction: " + str(getSentiment(predictionClass)))
if np.array_equal(groundTruth,prediction):
    print("\nPrediction is Correct")
else:
    print("\nPrediction is Incorrect")

Testing for test case...120
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
893
Text: what is the ground transportation between the charlotte airport charlotte airport and downtown charlotte 

Prediction: flight

Prediction is Correct


# Explainability

## Get Relevance for each word

In [43]:
def getWordsAndRelevances(sampleX, relevances, prediction):

    unknownIndex = 0 # Index of padding
    indexList = np.ndarray.tolist(sampleX)[0]
    wordList = []
    wordRelevanceList = []
    polarityHighThreshold = 0.4
    polarityLowThreshold = 0.0
    
    # Find word-wize relevance and normalize
    wordRelevances = np.sum(relevances, -1)
    wordRelList = np.ndarray.tolist(wordRelevances)[0]
    
    for i in range(maxWords):
        index = indexList[i]
        relevance = wordRelList[i]

        if index is not unknownIndex:
            word = index2Word[index]
            blobText = TextBlob(word)
            polarity = blobText.sentiment.polarity
            prediction = int(prediction)
#             if prediction is 0 and polarity < -0.4 or prediction is 1 and polarity > 0.4:
            if abs(polarity) > polarityHighThreshold:
                if prediction is 0 and polarity < 0 or prediction is 1 and polarity > 0:
                    relevance = abs(polarity)
                elif prediction is 0 and polarity > 0 or prediction is 1 and polarity < 0:
                    relevance = -1 * abs(polarity)
                    
                print(word, polarity)
                print(word, relevance)
            elif abs(polarity) > polarityLowThreshold:
                if relevance < 0 and polarity > 0 or relevance > 0 and polarity < 0:
                    relevance = polarity
#                     print("Here...")
#                     print(word, polarity)
#                     print(word, relevance)

            wordList.append(index2Word[index])
            wordRelevanceList.append(relevance)

    
    normalizedRelevanceList = [float(rel)/max(map(lambda x: abs(x), wordRelevanceList)) for rel in wordRelevanceList]
    
    return wordList, wordRelevanceList, normalizedRelevanceList

def showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList):

    print("\nWord Relevances:\n")
    print("\nOriginal Relevance:\n")
    for i in range(len(wordList)):
        word = str(wordList[i])
        # originalRelevance = "{:8.2f}".format(wordRelevanceList[i])
        originalRelevance = wordRelevanceList[i]
        print ("\t\t\t" + str(originalRelevance) + "\t" + word)

    print("\nNormalized Relevance:\n")
    for i in range(len(wordList)):
        word = str(wordList[i])
        normalizedRelevance = "{:8.2f}".format(normalizedRelevanceList[i])
        print ("\t\t\t" + str(normalizedRelevance) + "\t" + word)
        

## Pre-process and get prediction

In [44]:
def removePunct(inputStr):
    
    # Remove punctuations.
    # punctuations = string.punctuation
    # inputStr = ''.join(ch for ch in inputStr if ch not in punctuations)
    inputStr = inputStr.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    inputList = inputStr.lower().split()
    return inputList

def removeStopWords(inputList):
    processedList = [word for word in inputList if word not in stopWords]
    return processedList

def preProcessQuery(inputStr):
    return removeStopWords(removePunct(inputStr))
#     return removePunct(inputStr)

def getIndexArray(inputStr):
    
    words = preProcessQuery(inputStr)
    # print(words)
    wordIndexList = np.array([word2Index[word] if word in word2Index else 0 for word in words])
    wordIndexArray = np.array([wordIndexList], np.int32)
    wordIndexArray = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)
    return wordIndexArray

def getPrediction(inputStr):
    
    wordIndexArray = getIndexArray(inputStr)
    predictionScore = model.predict(wordIndexArray[0:1], verbose=0)
    # print (predictionScore)
    prediction = model.predict_classes(wordIndexArray[0:1], verbose=0)
    return prediction, predictionScore

## Get Polarity from input text

In [47]:
inputStr = input("Enter a sentence to test polarity: ")
prediction, predictionScore = getPrediction(inputStr)
print("\nPrediction: " + str(getSentiment(prediction)))
print(predictionScore)

Enter a sentence to test polarity: what is the ground transportation between the charlotte airport charlotte airport and downtown charlotte 

Prediction: ground_service
[[9.9979645e-01 1.8280483e-06 9.4161987e-05 8.5571929e-07 1.6346843e-06
  7.5795317e-09 2.8288358e-09 1.7049688e-06 6.4401598e-07 1.1307772e-06
  2.4074222e-06 4.4448313e-07 3.9912618e-07 2.6985759e-07 1.2041010e-06
  2.0299922e-06 1.3638832e-07 2.9519583e-07 5.8270460e-05 1.1049394e-05
  1.2920888e-08 2.5852279e-07 4.2115960e-08 1.4866860e-08 2.5097666e-05
  1.3102528e-08]]


# Relevance and HeatMap

## Create DeepExplain Session to get word relevances

In [48]:
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    
    '''
    Need to reconstruct the graph in DeepExplain context, using the same weights.
    1. Get the input tensor
    2. Get embedding tensor
    3. Target the output of the last dense layer (pre-softmax)
    '''
    
    inputTensor = model.layers[0].input
    embeddingTensor = model.layers[0].output
    preSoftmax = model.layers[-2].output
    
    # Sample Data for attribution
    wordIndexArray = getIndexArray(inputStr)
    sampleX = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)
    
    # Perform Embedding Lookup
    getEmbeddingOutput = K.function([inputTensor],[embeddingTensor])
    embeddingOutput = getEmbeddingOutput([sampleX])[0]
    
    # Get Prediction for attribution
    prediction, predictionScore = getPrediction(inputStr)
    ys = np_utils.to_categorical(prediction, num_classes = nb_classes)
    
    #relevances = de.explain('grad*input', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('saliency', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('intgrad', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('deeplift', targetTensor * ys, inputTensor, sampleX)
    relevances = de.explain('elrp', preSoftmax * ys, embeddingTensor, embeddingOutput)
    #relevances = de.explain('occlusion', targetTensor * ys, inputTensor, sampleX)
    print ("Feature Relevances Generated...")

Feature Relevances Generated...


## Show Relevance

In [49]:
wordList, wordRelevanceList, normalizedRelevanceList = getWordsAndRelevances(sampleX, relevances, prediction[0])
showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList)


Word Relevances:


Original Relevance:

			-0.09830529987812042	what
			-0.16111251711845398	is
			-0.41282957792282104	the
			0.8693649768829346	ground
			0.7951118350028992	transportation
			-0.3441087603569031	between
			-0.14844770729541779	the
			-0.3049934506416321	charlotte
			0.8933058977127075	airport
			-0.26179003715515137	charlotte
			0.7258991599082947	airport
			0.0242861770093441	and
			-0.2759798765182495	downtown
			-0.12554866075515747	charlotte

Normalized Relevance:

			   -0.11	what
			   -0.18	is
			   -0.46	the
			    0.97	ground
			    0.89	transportation
			   -0.39	between
			   -0.17	the
			   -0.34	charlotte
			    1.00	airport
			   -0.29	charlotte
			    0.81	airport
			    0.03	and
			   -0.31	downtown
			   -0.14	charlotte


## Generate Heatmap

In [50]:
def rescale_score_by_abs (score, max_score, min_score):
    """
    rescale positive score to the range [0.5, 1.0], negative score to the range [0.0, 0.5],
    using the extremal scores max_score and min_score for normalization
    """
    
    # CASE 1: positive AND negative scores occur --------------------
    if max_score>0 and min_score<0:
    
        if max_score >= abs(min_score):   # deepest color is positive
            if score>=0:
                return 0.5 + 0.5*(score/max_score)
            else:
                return 0.5 - 0.5*(abs(score)/max_score)

        else:                             # deepest color is negative
            if score>=0:
                return 0.5 + 0.5*(score/abs(min_score))
            else:
                return 0.5 - 0.5*(score/min_score)   
    
    # CASE 2: ONLY positive scores occur -----------------------------       
    elif max_score>0 and min_score>=0: 
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5*(score/max_score)
    
    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score<=0 and min_score<0: 
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5*(score/min_score)    
  
      
def getRGB (c_tuple):
    return "#%02x%02x%02x"%(int(c_tuple[0]*255), int(c_tuple[1]*255), int(c_tuple[2]*255))

     
def span_word (word, score, colormap):
    return "<span style=\"background-color:"+getRGB(colormap(score))+"\">"+word+"</span>"


def html_heatmap (words, scores, cmap_name="bwr"):
    
    colormap  = plt.get_cmap(cmap_name)
     
    assert len(words)==len(scores)
    max_s     = max(scores)
    min_s     = min(scores)
    
    output_text = ""
    
    for idx, w in enumerate(words):
        score       = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, colormap) + " "

    return output_text + "\n"
            
relevanceScores = [-1.00 * scr for scr in wordRelevanceList]
htmlHeatMap = html_heatmap(wordList, relevanceScores)
print ("\nLRP heatmap:")    
display(HTML(htmlHeatMap))


LRP heatmap:


In [ ]:
for i, inputStr in enumerate(x_test):
    inputStr = str(getWordsFromIndexList(inputStr))
    prediction, predictionScore = getPrediction(inputStr)
    print('Prediction = ', str(getSentiment(prediction)))
    print('Actual = ',sentimentDict[y_test[i]])
    if str(getSentiment(prediction)) == sentimentDict[y_test[i]]:
        print('MATCH')
    else :
        print('NO MATCH')
    with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context

        '''
        Need to reconstruct the graph in DeepExplain context, using the same weights.
        1. Get the input tensor
        2. Get embedding tensor
        3. Target the output of the last dense layer (pre-softmax)
        '''

        inputTensor = model.layers[0].input
        embeddingTensor = model.layers[0].output
        preSoftmax = model.layers[-2].output

        # Sample Data for attribution
        wordIndexArray = getIndexArray(inputStr)
        sampleX = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)

        # Perform Embedding Lookup
        getEmbeddingOutput = K.function([inputTensor],[embeddingTensor])
        embeddingOutput = getEmbeddingOutput([sampleX])[0]

        # Get Prediction for attribution
        prediction, predictionScore = getPrediction(inputStr)
        ys = np_utils.to_categorical(prediction, num_classes = nb_classes)

        #relevances = de.explain('grad*input', targetTensor * ys, inputTensor, sampleX)
        #relevances = de.explain('saliency', targetTensor * ys, inputTensor, sampleX)
        #relevances = de.explain('intgrad', targetTensor * ys, inputTensor, sampleX)
        #relevances = de.explain('deeplift', targetTensor * ys, inputTensor, sampleX)
        relevances = de.explain('elrp', preSoftmax * ys, embeddingTensor, embeddingOutput)
        #relevances = de.explain('occlusion', targetTensor * ys, inputTensor, sampleX)
#         print ("Feature Relevances Generated...")

        wordList, wordRelevanceList, normalizedRelevanceList = getWordsAndRelevances(sampleX, relevances, prediction[0])
    #     showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList)

        relevanceScores = [-1.00 * scr for scr in wordRelevanceList]
        htmlHeatMap = html_heatmap(wordList, relevanceScores)
#         print ("\nLRP heatmap:")    
        display(HTML(htmlHeatMap))

Prediction =  flight
Actual =  flight
MATCH


Prediction =  airfare
Actual =  airfare
MATCH


Prediction =  flight
Actual =  flight
MATCH


Prediction =  flight
Actual =  flight
MATCH
Here...
first 0.25
first 0.25


Prediction =  flight
Actual =  flight
MATCH
Here...
first 0.25
first 0.25


...

Prediction =  airline
Actual =  airline
MATCH


Prediction =  flight
Actual =  flight
MATCH
latest 0.5
latest 0.41543006896972656


Prediction =  flight
Actual =  flight
MATCH


Prediction =  ground_service
Actual =  ground_service
MATCH
Here...
worth 0.3
worth 0.3


Prediction =  capacity
Actual =  capacity
MATCH
